# <span style='background:DarkOliveGreen'> Operando o IDIV com base em Médias Móveis (9x21) no Pandas </span> ![Pandinha](pandinha_deitado_12abr22.png "Pandinha")

* Este JNB é uma continuaçao de um estudo feito em um JNB anterior:  
https://rafsz.github.io/Python+Pandas21/MAs_9x21_23mar22_18h.html

* Este JNB tem 4 etapas:
1. Pegando as cotações de uma lista de ações até um dia D e guardando em um Pandas df
2. Calculando as Médias Móveis (MMs) e obtendo as mensagens de BUY (comprar), STOP(stopar, zerar a posição), Hold (segurar, manter a posição) ou Wait (aguardar, não comprar)
3. Inserindo as mensagens de BUY, STOP, Hold ou WAIT no df original e salvando no excel

<div class="alert alert-warning">
  <strong>  I. Pegando as cotações e guardando em um Pandas df </strong>
</div>

In [1]:
# importando as Bibliotecas que vamos usar
import yfinance as yf
import pandas as pd

In [2]:
# Neste JNB vamos usar uma lista de acoes tiradas do IDIV - o índice de boas pagadoras de dividendos da B3 
# O arq excel com a composição deste índice pode ser baixado em:  
# https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-dividendos-idiv-composicao-da-carteira.htm
# Na composição do IDIV de 22.mar.22 consta o papel AURE3, que veio substituir a CESP
# https://www.infomoney.com.br/cotacoes/b3/acao/auren-energia-aure3/
# Como o histórico deste papel começa só em 28/03, não temos com calcular a sua média móvel de 21 dias ainda...  Portanto, ele foi retirado da nossa lista.

In [3]:
# O arq CSV baixado do site da B3 foi editado no LibreOffice e salvo como xlsx (Excel)
# Lendo o arq excel simplificado com 3 colunas com a composicao do IDIV e salvando num Pandas df
dfidiva = pd.read_excel("B3_IDIV_31-03-22-editadosemAURE3.xlsx")

In [4]:
# visualizando o df
display (dfidiva)

,Código,Ação,Part. (%)
0,ABCB4,ABC BRASIL,0.597
1,BRSR6,BANRISUL,1.149
2,BBSE3,BBSEGURIDADE,3.669
3,BBDC3,BRADESCO,3.627
4,BBDC4,BRADESCO,3.736
5,BRAP4,BRADESPAR,4.174
6,BBAS3,BRASIL,3.306
7,AGRO3,BRASILAGRO,0.834
8,CCRO3,CCR SA,2.765
9,CMIG3,CEMIG,2.745


In [5]:
# transformando a Coluna de Tickers em uma Lista de Tickers
# https://www.delftstack.com/howto/python-pandas/pandas-column-to-list/
lidiva = dfidiva['Código'].tolist()
print(lidiva)
print(len(lidiva))

['ABCB4', 'BRSR6', 'BBSE3', 'BBDC3', 'BBDC4', 'BRAP4', 'BBAS3', 'AGRO3', 'CCRO3', 'CMIG3', 'CMIG4', 'CSMG3', 'CPLE3', 'CPLE6', 'CPFE3', 'CYRE3', 'DIRR3', 'ELET3', 'ELET6', 'ENAT3', 'ENBR3', 'EGIE3', 'ROMI3', 'MYPK3', 'ITSA4', 'ITUB3', 'ITUB4', 'JHSF3', 'MRVE3', 'PSSA3', 'QUAL3', 'SAPR4', 'SANB11', 'CSNA3', 'SYNE3', 'TAEE11', 'TGMA3', 'VIVT3', 'TRPL4', 'UNIP6', 'WIZS3']
41


In [6]:
# criando uma nova lista para ter o histórico de mudanças
# inserindo a terminacao .SA que o YFinance pede
lidivb = []
for ticker in lidiva:
    lidivb.append(ticker+'.SA')
print(lidivb)
print(len(lidivb))

['ABCB4.SA', 'BRSR6.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA', 'BBAS3.SA', 'AGRO3.SA', 'CCRO3.SA', 'CMIG3.SA', 'CMIG4.SA', 'CSMG3.SA', 'CPLE3.SA', 'CPLE6.SA', 'CPFE3.SA', 'CYRE3.SA', 'DIRR3.SA', 'ELET3.SA', 'ELET6.SA', 'ENAT3.SA', 'ENBR3.SA', 'EGIE3.SA', 'ROMI3.SA', 'MYPK3.SA', 'ITSA4.SA', 'ITUB3.SA', 'ITUB4.SA', 'JHSF3.SA', 'MRVE3.SA', 'PSSA3.SA', 'QUAL3.SA', 'SAPR4.SA', 'SANB11.SA', 'CSNA3.SA', 'SYNE3.SA', 'TAEE11.SA', 'TGMA3.SA', 'VIVT3.SA', 'TRPL4.SA', 'UNIP6.SA', 'WIZS3.SA']
41


In [8]:
# Pegando as cotacoes de Fechamento Ajustado e guardando num df
# Queremos do início do ano até 6a-feira 01/04, então a data final vai ser o dia seguinte
dfidivb = yf.download(lidivb, start="2022-01-01", end = "2022-04-02")["Adj Close"]
dfidivb.shape

[*********************100%***********************]  41 of 41 completed


(64, 41)

In [9]:
# Verificando se tem algum valor nulo no df
dfidivb.isna().sum()

ABCB4.SA     1
AGRO3.SA     1
BBAS3.SA     1
BBDC3.SA     1
BBDC4.SA     1
BBSE3.SA     1
BRAP4.SA     1
BRSR6.SA     1
CCRO3.SA     1
CMIG3.SA     1
CMIG4.SA     1
CPFE3.SA     1
CPLE3.SA     1
CPLE6.SA     1
CSMG3.SA     1
CSNA3.SA     1
CYRE3.SA     1
DIRR3.SA     1
EGIE3.SA     1
ELET3.SA     1
ELET6.SA     1
ENAT3.SA     1
ENBR3.SA     1
ITSA4.SA     1
ITUB3.SA     1
ITUB4.SA     1
JHSF3.SA     1
MRVE3.SA     1
MYPK3.SA     1
PSSA3.SA     1
QUAL3.SA     1
ROMI3.SA     1
SANB11.SA    1
SAPR4.SA     1
SYNE3.SA     1
TAEE11.SA    1
TGMA3.SA     1
TRPL4.SA     1
UNIP6.SA     1
VIVT3.SA     1
WIZS3.SA     1
dtype: int64

In [10]:
# Verificando quais são as datas que têm valores nulos
nan_values = dfidivb[dfidivb.isna().any(axis=1)]
print (nan_values)

            ABCB4.SA  AGRO3.SA  BBAS3.SA  BBDC3.SA  BBDC4.SA  BBSE3.SA  \
Date                                                                     
2022-03-01       NaN       NaN       NaN       NaN       NaN       NaN   

            BRAP4.SA  BRSR6.SA  CCRO3.SA  CMIG3.SA  ...  ROMI3.SA  SANB11.SA  \
Date                                                ...                        
2022-03-01       NaN       NaN       NaN       NaN  ...       NaN        NaN   

            SAPR4.SA  SYNE3.SA  TAEE11.SA  TGMA3.SA  TRPL4.SA  UNIP6.SA  \
Date                                                                      
2022-03-01       NaN       NaN        NaN       NaN       NaN       NaN   

            VIVT3.SA  WIZS3.SA  
Date                            
2022-03-01       NaN       NaN  

[1 rows x 41 columns]


In [11]:
# O valor com NaN é 01/03/22, 3a de Carnaval
# Vamos eliminar os valores nulos e verificar se o "shape" caiu para (63, 41) (isto é, 63 linhas, 41 colunas, uma para cada ação)
dfidivb = dfidivb.dropna(axis = 0, how ='any')
dfidivb.shape

(63, 41)

<div class="alert alert-warning">
  <strong>  II. Calculando as Médias Móveis (MMs) e obtendo as mensagens de BUY, STOP, Hold ou Wait </strong>
</div>

In [12]:
# Vamos calcular as Médias Móveis (MM) e substituir os valores das cotações pela diferença da MM09 para a MM21
# Esta diferença vai nos indicar se a MM09 está acima (valor positivo) da MM21 ou abaixo (valor negativo)
for (col) in dfidivb:
    dfidivb[col] = (dfidivb[col].rolling(9).mean()) - (dfidivb[col].rolling(21).mean())

In [14]:
dfidivb.head(30)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CMIG3.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Verificando se tem algum valor nulo no df
dfidivb.isna().sum()

ABCB4.SA     20
AGRO3.SA     20
BBAS3.SA     20
BBDC3.SA     20
BBDC4.SA     20
BBSE3.SA     20
BRAP4.SA     20
BRSR6.SA     20
CCRO3.SA     20
CMIG3.SA     20
CMIG4.SA     20
CPFE3.SA     20
CPLE3.SA     20
CPLE6.SA     20
CSMG3.SA     20
CSNA3.SA     20
CYRE3.SA     20
DIRR3.SA     20
EGIE3.SA     20
ELET3.SA     20
ELET6.SA     20
ENAT3.SA     20
ENBR3.SA     20
ITSA4.SA     20
ITUB3.SA     20
ITUB4.SA     20
JHSF3.SA     20
MRVE3.SA     20
MYPK3.SA     20
PSSA3.SA     20
QUAL3.SA     20
ROMI3.SA     20
SANB11.SA    20
SAPR4.SA     20
SYNE3.SA     20
TAEE11.SA    20
TGMA3.SA     20
TRPL4.SA     20
UNIP6.SA     20
VIVT3.SA     20
WIZS3.SA     20
dtype: int64

In [ ]:
# Ou seja, as 20 primieras datas estão com NaN uma vez que a MM21 não pode ser calculada

In [18]:
# Eliminando as linhas NA do novo df
dfidivb = dfidivb.dropna(axis = 0, how ='any')
dfidivb.head(10)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CMIG3.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-21,0.187172,0.440808,-0.178100,0.116757,0.315359,-0.014286,1.762857,0.163944,0.103175,0.452086,...,-0.644569,1.578730,-0.008889,-0.049048,1.121905,0.038095,0.196190,4.051429,1.080141,0.123175
2022-03-22,0.254498,0.330233,-0.131690,0.183137,0.389433,0.130158,1.636031,0.193526,0.170635,0.560184,...,-0.496404,1.823810,-0.017937,-0.029048,1.074285,-0.081587,0.237460,4.271587,1.334361,0.259524
2022-03-23,0.337720,0.179312,-0.013748,0.264737,0.474763,0.253174,1.407143,0.251792,0.262857,0.690342,...,-0.371892,2.090794,-0.013968,0.009365,1.031428,-0.152698,0.326667,4.211111,1.604908,0.365079
2022-03-24,0.397409,-0.010310,0.169089,0.362050,0.578500,0.403333,1.203809,0.302956,0.436667,0.835627,...,-0.174526,2.275397,-0.009683,0.064762,1.099524,-0.082381,0.430000,4.258413,1.820207,0.483810
2022-03-25,0.432163,-0.175426,0.342719,0.455716,0.674470,0.561905,1.066666,0.371238,0.643810,1.001712,...,0.037523,2.379841,-0.000159,0.097302,1.252857,0.037778,0.563968,4.286826,1.967811,0.588730
2022-03-28,0.542502,-0.224588,0.545233,0.544318,0.764259,0.723333,0.929365,0.441032,0.824286,1.163071,...,0.236480,2.489048,0.015714,0.157302,1.396508,0.172222,0.723968,4.116826,2.031355,0.690317
2022-03-29,0.591906,-0.219657,0.655700,0.596783,0.816172,0.855079,0.669364,0.472938,0.920000,1.261310,...,0.418915,2.447302,0.026667,0.233810,1.476508,0.297143,0.820635,3.659524,2.043068,0.760952
2022-03-30,0.603750,-0.323807,0.744267,0.632118,0.855076,0.944286,0.373174,0.490880,0.926667,1.286874,...,0.522029,2.380952,0.034921,0.296190,1.488413,0.439524,0.879365,2.946031,2.002511,0.793810
2022-03-31,0.554970,-0.406590,0.845937,0.652553,0.876227,1.073333,0.089365,0.495737,0.922064,1.329191,...,0.555692,2.263492,0.046032,0.303333,1.484762,0.494127,0.912540,2.311904,1.857835,0.780635


In [19]:
dfidivb.tail(10)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CMIG3.SA,...,ROMI3.SA,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-21,0.187172,0.440808,-0.178100,0.116757,0.315359,-0.014286,1.762857,0.163944,0.103175,0.452086,...,-0.644569,1.578730,-0.008889,-0.049048,1.121905,0.038095,0.196190,4.051429,1.080141,0.123175
2022-03-22,0.254498,0.330233,-0.131690,0.183137,0.389433,0.130158,1.636031,0.193526,0.170635,0.560184,...,-0.496404,1.823810,-0.017937,-0.029048,1.074285,-0.081587,0.237460,4.271587,1.334361,0.259524
2022-03-23,0.337720,0.179312,-0.013748,0.264737,0.474763,0.253174,1.407143,0.251792,0.262857,0.690342,...,-0.371892,2.090794,-0.013968,0.009365,1.031428,-0.152698,0.326667,4.211111,1.604908,0.365079
2022-03-24,0.397409,-0.010310,0.169089,0.362050,0.578500,0.403333,1.203809,0.302956,0.436667,0.835627,...,-0.174526,2.275397,-0.009683,0.064762,1.099524,-0.082381,0.430000,4.258413,1.820207,0.483810
2022-03-25,0.432163,-0.175426,0.342719,0.455716,0.674470,0.561905,1.066666,0.371238,0.643810,1.001712,...,0.037523,2.379841,-0.000159,0.097302,1.252857,0.037778,0.563968,4.286826,1.967811,0.588730
2022-03-28,0.542502,-0.224588,0.545233,0.544318,0.764259,0.723333,0.929365,0.441032,0.824286,1.163071,...,0.236480,2.489048,0.015714,0.157302,1.396508,0.172222,0.723968,4.116826,2.031355,0.690317
2022-03-29,0.591906,-0.219657,0.655700,0.596783,0.816172,0.855079,0.669364,0.472938,0.920000,1.261310,...,0.418915,2.447302,0.026667,0.233810,1.476508,0.297143,0.820635,3.659524,2.043068,0.760952
2022-03-30,0.603750,-0.323807,0.744267,0.632118,0.855076,0.944286,0.373174,0.490880,0.926667,1.286874,...,0.522029,2.380952,0.034921,0.296190,1.488413,0.439524,0.879365,2.946031,2.002511,0.793810
2022-03-31,0.554970,-0.406590,0.845937,0.652553,0.876227,1.073333,0.089365,0.495737,0.922064,1.329191,...,0.555692,2.263492,0.046032,0.303333,1.484762,0.494127,0.912540,2.311904,1.857835,0.780635


In [20]:
# Agora que temos o df com a indicação por data se cada papel estava "cruzado pra cima" ou "cruzado pra baixo", podemos obter as mensagens de Compra, Venda, etc.

In [21]:
# Funções para "printar" as mensagens coloridas aqui na tela do JNB
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk))
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk))
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk))

In [22]:
# A Função principal: que vai nos mostrar a mensagem de acordo com a sitaução das MMs de cada papel, comparando D e D-1
# Vou chamar a atenção pras mensagens BUY e STOP, que são as mais importantes
def alertacolor (acao):
    if (dfidivb.at[dfidivb.index[a],acao] > 0) and (dfidivb.at[dfidivb.index[b],acao] < 0):
        prGreen('  B-U-Y  !!!! ')
    elif (dfidivb.at[dfidivb.index[a],acao] < 0) and (dfidivb.at[dfidivb.index[b],acao] > 0):
        prRed('  StoOoOop!!')
    elif (dfidivb.at[dfidivb.index[a],acao] > 0) and (dfidivb.at[dfidivb.index[b],acao] > 0):
        prGreen('  hold ')
    else:
        prYellow('  wait...')

In [24]:
# Chamando esta função para a nossa lista de papéis
for papel in lidivb:
    a = -1
    b = a-1
    prCyan (papel+': ')
    alertacolor(papel)
    print(' ')

 ABCB4.SA: 
   hold 
 
 BRSR6.SA: 
   hold 
 
 BBSE3.SA: 
   hold 
 
 BBDC3.SA: 
   hold 
 
 BBDC4.SA: 
   hold 
 
 BRAP4.SA: 
   StoOoOop!!
 
 BBAS3.SA: 
   hold 
 
 AGRO3.SA: 
   wait...
 
 CCRO3.SA: 
   hold 
 
 CMIG3.SA: 
   hold 
 
 CMIG4.SA: 
   hold 
 
 CSMG3.SA: 
   hold 
 
 CPLE3.SA: 
   hold 
 
 CPLE6.SA: 
   hold 
 
 CPFE3.SA: 
   hold 
 
 CYRE3.SA: 
   hold 
 
 DIRR3.SA: 
   hold 
 
 ELET3.SA: 
   hold 
 
 ELET6.SA: 
   hold 
 
 ENAT3.SA: 
   hold 
 
 ENBR3.SA: 
   hold 
 
 EGIE3.SA: 
   hold 
 
 ROMI3.SA: 
   hold 
 
 MYPK3.SA: 
   hold 
 
 ITSA4.SA: 
   hold 
 
 ITUB3.SA: 
   hold 
 
 ITUB4.SA: 
   hold 
 
 JHSF3.SA: 
   hold 
 
 MRVE3.SA: 
   hold 
 
 PSSA3.SA: 
   hold 
 
 QUAL3.SA: 
   hold 
 
 SAPR4.SA: 
   hold 
 
 SANB11.SA: 
   hold 
 
 CSNA3.SA: 
   wait...
 
 SYNE3.SA: 
   hold 
 
 TAEE11.SA: 
   hold 
 
 TGMA3.SA: 
   hold 
 
 VIVT3.SA: 
   hold 
 
 TRPL4.SA: 
   hold 
 
 UNIP6.SA: 
   hold 
 
 WIZS3.SA: 
   hold 
 


<div class="alert alert-warning">
  <strong>III. Inserindo as mensagens de BUY, STOP, Hold ou Wait no df original e salvando no formato excel</strong>
</div>

In [26]:
# Vamos criar uma funcao similar à "alertacolour" somente para obter as mensagens
def alertasimples (acao):
    if (dfidivb.at[dfidivb.index[a],acao] > 0) and (dfidivb.at[dfidivb.index[b],acao] < 0):
        return('BUY')
    elif (dfidivb.at[dfidivb.index[a],acao] < 0) and (dfidivb.at[dfidivb.index[b],acao] > 0):
        return('STOP!')
    elif (dfidivb.at[dfidivb.index[a],acao] > 0) and (dfidivb.at[dfidivb.index[b],acao] > 0):
        return('hold')
    else:
        return('wait')

In [27]:
# Vamos usar a nova função pra criar um dicionário que casa os codigos com as mensagens
# https://www.geeksforgeeks.org/use-get-method-to-create-a-dictionary-in-python-from-a-list-of-elements/?ref=gcse
dcidiva={}
for papel in lidivb:
    a = -1
    b = a-1
    dcidiva[papel]=alertasimples(papel)
print(dcidiva)

{'ABCB4.SA': 'hold', 'BRSR6.SA': 'hold', 'BBSE3.SA': 'hold', 'BBDC3.SA': 'hold', 'BBDC4.SA': 'hold', 'BRAP4.SA': 'STOP!', 'BBAS3.SA': 'hold', 'AGRO3.SA': 'wait', 'CCRO3.SA': 'hold', 'CMIG3.SA': 'hold', 'CMIG4.SA': 'hold', 'CSMG3.SA': 'hold', 'CPLE3.SA': 'hold', 'CPLE6.SA': 'hold', 'CPFE3.SA': 'hold', 'CYRE3.SA': 'hold', 'DIRR3.SA': 'hold', 'ELET3.SA': 'hold', 'ELET6.SA': 'hold', 'ENAT3.SA': 'hold', 'ENBR3.SA': 'hold', 'EGIE3.SA': 'hold', 'ROMI3.SA': 'hold', 'MYPK3.SA': 'hold', 'ITSA4.SA': 'hold', 'ITUB3.SA': 'hold', 'ITUB4.SA': 'hold', 'JHSF3.SA': 'hold', 'MRVE3.SA': 'hold', 'PSSA3.SA': 'hold', 'QUAL3.SA': 'hold', 'SAPR4.SA': 'hold', 'SANB11.SA': 'hold', 'CSNA3.SA': 'wait', 'SYNE3.SA': 'hold', 'TAEE11.SA': 'hold', 'TGMA3.SA': 'hold', 'VIVT3.SA': 'hold', 'TRPL4.SA': 'hold', 'UNIP6.SA': 'hold', 'WIZS3.SA': 'hold'}


In [28]:
# Vamos inserir uma coluna com os codigos.SA no df inicial (dfidiva) para poder casar com o dicionario e poder importar as mensagens
# https://www.statology.org/pandas-combine-two-columns/
dfidiva['codigoSA'] = dfidiva['Código'] + '.SA'
display(dfidiva)

,Código,Ação,Part. (%),codigoSA
0,ABCB4,ABC BRASIL,0.597,ABCB4.SA
1,BRSR6,BANRISUL,1.149,BRSR6.SA
2,BBSE3,BBSEGURIDADE,3.669,BBSE3.SA
3,BBDC3,BRADESCO,3.627,BBDC3.SA
4,BBDC4,BRADESCO,3.736,BBDC4.SA
5,BRAP4,BRADESPAR,4.174,BRAP4.SA
6,BBAS3,BRASIL,3.306,BBAS3.SA
7,AGRO3,BRASILAGRO,0.834,AGRO3.SA
8,CCRO3,CCR SA,2.765,CCRO3.SA
9,CMIG3,CEMIG,2.745,CMIG3.SA


In [32]:
# https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
# Agora que temos uma coluna com os codigos dos papeis identicos as chaves do dicionario,
# Podemos criar uma coluna cujos valores serão aqueles dos dicionario "dcidiva" (ie, as mensagens de BUY, STOP, etc)
dfidiva['2022-04-01'] = dcidiva.values()
display(dfidiva)

,Código,Ação,Part. (%),codigoSA,2022-04-01
0,ABCB4,ABC BRASIL,0.597,ABCB4.SA,hold
1,BRSR6,BANRISUL,1.149,BRSR6.SA,hold
2,BBSE3,BBSEGURIDADE,3.669,BBSE3.SA,hold
3,BBDC3,BRADESCO,3.627,BBDC3.SA,hold
4,BBDC4,BRADESCO,3.736,BBDC4.SA,hold
5,BRAP4,BRADESPAR,4.174,BRAP4.SA,STOP!
6,BBAS3,BRASIL,3.306,BBAS3.SA,hold
7,AGRO3,BRASILAGRO,0.834,AGRO3.SA,wait
8,CCRO3,CCR SA,2.765,CCRO3.SA,hold
9,CMIG3,CEMIG,2.745,CMIG3.SA,hold


In [33]:
# Nós vamos manter a coluna CódigoSA pq ela vai ser útil para inserir mensagens para os pregões seguintes...
# Vamos agora salvar este df em excel
# https://cursos.alura.com.br/forum/topico-salvar-arquivo-excel-com-varias-abas-145704
writer = pd.ExcelWriter('B3_IDIV_em_2022-04-01.xlsx', engine='xlsxwriter')
# Armazena cada df em uma planilha diferente do mesmo arquivo
dfidiva.to_excel(writer, sheet_name='IDIV_01abr22')
# Fecha o ExcelWriter e gera o arquivo .xlsx
writer.save()

### Visualizando o arq Excel:

<img src="Pandas_df_xlsx_01abr22.jpg" alt="MensagensnoExcel"/>

#### fim  🐍 

  - Referências:
  
https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-dividendos-idiv-composicao-da-carteira.htm

Canal: Código Quant - Finanças Quantitativas
PYTHON PARA INVESTIMENTOS #2: Definindo intervalos, calculando e plotando médias móveis  
https://youtu.be/BBomKv3NFNc?list=PLmQ5Q79miLmxDc16motuYuG1hLjW0T4Wo

Inserir uma nova coluna num df e copiar os valores de um dicionario pra esta coluna, contanto que as chaves do dicionario batam com alguma coluna do df:
https://www.geeksforgeeks.org/use-get-method-to-create-a-dictionary-in-python-from-a-list-of-elements/?ref=gcse

Para consulta geral:  
https://betterprogramming.pub/9-pandas-functions-that-will-do-99-of-any-analytics-task-e6b6fb1b16bf


